In [1]:
import os
import librosa
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
#Loading training and testing files
#Computing STFT on all the files
s, sr = librosa.load('train_clean_male.wav', sr=None)
S = librosa.stft(s, n_fft=1024, hop_length=512)

sn, sr = librosa.load('train_dirty_male.wav', sr=None)
X = librosa.stft(sn, n_fft=1024, hop_length=512)

x_test, sr = librosa.load('test_01.wav', sr=None)
X_test = librosa.stft(x_test, n_fft=1024, hop_length=512)

x_test2, sr = librosa.load('test_02.wav', sr=None)
X_test2 = librosa.stft(x_test2, n_fft=1024, hop_length=512)

In [3]:
#Calculating the magnitude of all the input files
mag_S = np.abs(S)
mag_X = np.abs(X)
mag_X_test = np.abs(X_test)
mag_X_test2 = np.abs(X_test2)

#Defining model specifications
learning_rate = 0.001
act_layers = [tf.nn.relu, tf.nn.relu, tf.nn.relu, tf.nn.relu]
neurons = [513, 513, 513, 513]
num_layers = len(act_layers)

In [4]:
def getModel(x , act_layers , neurons):
    num_layers = len(act_layers)
    layers = [0]*num_layers

    for i in range(0 , len(act_layers)):
        if i == 0:
            layers[i] = tf.keras.layers.Dense(units=neurons[i] , activation=act_layers[i]) # Removed 'x' and pass input when calling the layer
        elif i < num_layers-1:
            layers[i] = tf.keras.layers.Dense(units=neurons[i] , activation=act_layers[i]) # Removed 'layers[i-1]' and will pass input during call
        else:
            layers[i] = tf.keras.layers.Dense(units=neurons[i] , activation=act_layers[i]) # Removed 'layers[i-1]' and will pass input during call

    #Chain the layers together by applying them sequentially to the input
    output = x
    for layer in layers:
        output = layer(output)
    return output # Return the output of the final layer

In [5]:
import tensorflow as tf

# Hyperparameters
learning_rate = 0.0005
batch_size = 100
epochs = 200

# Placeholder data: Replace these with your actual data tensors
# Assuming mag_X and mag_S are numpy arrays or tensors of shape (features, samples)
mag_X = tf.random.normal([100, 2459])  # Example input: Replace with actual data
mag_S = tf.random.normal([100, 2459])  # Example target: Replace with actual data

# Model definition
num_features = mag_X.shape[0]  # Number of features
input_layer = tf.keras.layers.Input(shape=(num_features,), name="input")
hidden_layer1 = tf.keras.layers.Dense(128, activation="relu", name="hidden1")(input_layer)
hidden_layer2 = tf.keras.layers.Dense(256, activation="relu", name="hidden2")(hidden_layer1)
hidden_layer3 = tf.keras.layers.Dense(128, activation="relu", name="hidden3")(hidden_layer2)
output_layer = tf.keras.layers.Dense(num_features, activation=None, name="output")(hidden_layer3)

model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
optimizer = tf.keras.optimizers.Adam(learning_rate)

# Loss function
def loss_function(labels, predictions):
    return tf.reduce_mean(tf.square(labels - predictions))

# Training loop
for epoch in range(epochs):
    size = 0
    for i in range(0, 2500, batch_size):
        size += batch_size
        if size <= 2459:
            batch_x = mag_X[:, i:size]
            batch_y = mag_S[:, i:size]
        else:
            batch_x = mag_X[:, i:2459]
            batch_y = mag_S[:, i:2459]

        # Gradient computation and weight updates
        with tf.GradientTape() as tape:
            # Transpose using tf.transpose()
            predictions = model(tf.transpose(batch_x), training=True)
            loss_value = loss_function(tf.transpose(batch_y), predictions)

        gradients = tape.gradient(loss_value, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    # Log loss every 10 epochs
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss_value.numpy()}")

print("Training complete!")


Epoch 0, Loss: 1.0077227354049683
Epoch 10, Loss: 0.9415057301521301
Epoch 20, Loss: 0.8286869525909424
Epoch 30, Loss: 0.7275855541229248
Epoch 40, Loss: 0.6518456935882568
Epoch 50, Loss: 0.5973954796791077
Epoch 60, Loss: 0.5590300559997559
Epoch 70, Loss: 0.5329704284667969
Epoch 80, Loss: 0.5077674984931946
Epoch 90, Loss: 0.48839032649993896
Epoch 100, Loss: 0.47629687190055847
Epoch 110, Loss: 0.46272584795951843
Epoch 120, Loss: 0.44609010219573975
Epoch 130, Loss: 0.4435802698135376
Epoch 140, Loss: 0.43001654744148254
Epoch 150, Loss: 0.4266510605812073
Epoch 160, Loss: 0.41732853651046753
Epoch 170, Loss: 0.4113408625125885
Epoch 180, Loss: 0.4072929322719574
Epoch 190, Loss: 0.40178999304771423
Training complete!


In [6]:
#Calculating the output from the given input, trained model and layer number
import tensorflow as tf
import numpy as np

# Feedforward function
def feedforward(input_data, model, layer_num):
    # Ensure the input_data is a numpy array or tensor compatible with model
    input_tensor = tf.convert_to_tensor(input_data, dtype=tf.float32)
    intermediate_model = tf.keras.Model(inputs=model.input, outputs=model.layers[layer_num].output)
    output = intermediate_model(input_tensor)

    return output.numpy()

# Recovering the complex values of the file from the output of the model
def recover_sound(X, mag_X, mag_output):
    # Assuming X is the original signal and mag_X is the magnitude spectrogram
    # Recovering the complex value by applying the predicted magnitude with original phase
    phase_X = np.angle(X)  # Get the phase of X

    # Pad mag_output to match X's shape using tf.pad()
    pad_rows = X.shape[0] - mag_output.shape[0]
    pad_cols = X.shape[1] - mag_output.shape[1]

    # Ensure pad_rows and pad_cols are non-negative
    pad_rows = max(0, pad_rows)
    pad_cols = max(0, pad_cols)

    mag_output = tf.pad(mag_output, [[0, pad_rows], [0, pad_cols]])

    # Convert mag_output to NumPy array to perform multiplication
    mag_output = mag_output.numpy()

    recovered_complex = mag_output * np.exp(1j * phase_X)  # Recombine magnitude with original phase

    # Perform an inverse STFT to recover the time-domain signal (if using STFT)
    s_hat = np.fft.ifft(recovered_complex).real  # Assuming s_hat is in time-domain

    return s_hat



In [7]:
model
model.summary()



Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input (InputLayer)                   │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ hidden1 (Dense)                      │ (None, 128)                 │          12,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ hidden2 (Dense)                      │ (None, 256)                 │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ hidden3 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output (Dense)                       │ (None, 100)                 │          12,900 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 91,748 (358.39 KB)

 Trainable params: 91,748 (358.39 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
# Get the model's output for test_01.wav
# Calculate the number of features needed to maintain the total elements
num_features_test1 = mag_X_test.shape[0]  # Get the original number of features
num_samples_test1 = mag_X_test.shape[1]  # Get the number of samples

# Transpose to match the expected input shape (None, 100)
# The input shape should be (num_samples_test1, num_features)
mag_X_test_reshaped = tf.transpose(mag_X_test)
# Instead of padding, truncate the features to 100 if necessary
mag_X_test_reshaped = mag_X_test_reshaped[:, :100]  # Truncate to 100 features

# Predict using the model
mag_output_test1 = model.predict(mag_X_test_reshaped)

# Recover the sound from the model's output
s_hat_test1 = recover_sound(X_test, mag_X_test, tf.transpose(mag_output_test1))

5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step


In [9]:
## Computing the output from the model for both the test files
# Assuming mag_X_test and mag_X_test2 are the processed data
# Transpose and truncate mag_X_test to match the expected input shape
mag_X_test_input = tf.transpose(mag_X_test)[:, :100]
mag_X_test2_input = tf.transpose(mag_X_test2)[:, :100]

s_hat_test1 = feedforward(mag_X_test_input, model, 2)
s_hat_test2 = feedforward(mag_X_test2_input, model, 2)

# Recovering the complex values of both the test files
s_hat1 = recover_sound(X_test, mag_X_test, s_hat_test1.T)
s_hat2 = recover_sound(X_test2, mag_X_test2, s_hat_test2.T)

# Reconstructing the test files after removing noise
import librosa
import soundfile as sf # Make sure to import soundfile

recon_sound1 = librosa.istft(s_hat1, hop_length=512, win_length=1024)
sf.write('test_s_01_recons.wav', recon_sound1, sr)
print("Reconstructed test_s_01_recons.wav saved successfully.")

recon_sound2 = librosa.istft(s_hat2, hop_length=512, win_length=1024)
sf.write('test_s_02_recons.wav', recon_sound2, sr)
print("Reconstructed test_s_02_recons.wav saved successfully.")

Reconstructed test_s_01_recons.wav saved successfully.
Reconstructed test_s_02_recons.wav saved successfully.


In [10]:
#For testing purpose, feeding the model with train_dirty_male file
#From the output generated, reconstructing the audio file
s_hat_test3 = feedforward(tf.transpose(mag_X) , model , 2) # Changed mag_X.T to tf.transpose(mag_X) and output to model, 4 to 2
s_hat3 = recover_sound(X, mag_X , s_hat_test3.T)
recon_sound3 = librosa.istft(s_hat3 , hop_length=512 , win_length=1024)
size_recon_sound3 = np.shape(recon_sound3)[0]

In [11]:
#Once the audio file is generated, calculating the SNR value
s = s[: size_recon_sound3]
num = np.dot(s.T , s)
den = np.dot((s - recon_sound3).T,(s - recon_sound3))
SNR = 10 * np.log10(num/den)
print('Value of SNR : ' + str(SNR))

Value of SNR : -0.0010992574
